In [3]:
import pandas as pd
from datasets import Dataset



# Carica il file CSV
df = pd.read_csv('.\data\predictive_maintenance.csv')

# Rimuovi le colonne 'Target' e 'Failure Type'
df = df.drop(columns=['Target', 'Failure Type'])

# Crea un dataset Hugging Face
dataset = Dataset.from_pandas(df)


c:\Users\luca-\OneDrive\Desktop\University\#MAGISTRALE\2 ANNO\1 SEMESTRE\DEEP LEARNING\project\informer_project_dl\.venv\lib\site-packages\pyarrow\pandas_compat.py:571: UserWarning: pyarrow requires pandas 1.0.0 or above, pandas 0.25.1 is installed. Therefore, pandas-specific integration is not used.
  columns)


ImportError: pyarrow requires pandas 1.0.0 or above, pandas 0.25.1 is installed

In [ ]:
from transformers import InformerConfig

config = InformerConfig(
    prediction_length=24,  # Numero di step futuri da prevedere
    context_length=48,     # Numero di step passati da considerare
    num_time_features=2,   # Numero di caratteristiche temporali
    num_static_real_features=6,  # Numero di caratteristiche statiche reali
    lags_sequence=[1, 2, 3, 4, 5, 6, 7]  # Sequenza di lag
)

from transformers import InformerForPrediction

model = InformerForPrediction(config)

import torch
from torch.utils.data import DataLoader

# Definisci una funzione per creare sequenze di dati
def create_sequences(dataset, context_length, prediction_length):
    sequences = []
    for i in range(len(dataset) - context_length - prediction_length):
        past = dataset[i:i + context_length]
        future = dataset[i + context_length:i + context_length + prediction_length]
        sequences.append((past, future))
    return sequences

# Crea le sequenze
sequences = create_sequences(dataset, config.context_length, config.prediction_length)

# Crea un DataLoader
dataloader = DataLoader(sequences, batch_size=32, shuffle=True)

import torch
from torch.utils.data import DataLoader

# Definisci una funzione per creare sequenze di dati
def create_sequences(dataset, context_length, prediction_length):
    sequences = []
    for i in range(len(dataset) - context_length - prediction_length):
        past = dataset[i:i + context_length]
        future = dataset[i + context_length:i + context_length + prediction_length]
        sequences.append((past, future))
    return sequences

# Crea le sequenze
sequences = create_sequences(dataset, config.context_length, config.prediction_length)

# Crea un DataLoader
dataloader = DataLoader(sequences, batch_size=32, shuffle=True)

from transformers import InformerForPrediction
import torch.optim as optim

# Inizializza il modello
model = InformerForPrediction(config)

# Definisci l'ottimizzatore
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definisci la funzione di perdita
loss_fn = torch.nn.MSELoss()

# Ciclo di addestramento
model.train()
for epoch in range(10):  # Numero di epoche
    total_loss = 0
    for past, future in dataloader:
        optimizer.zero_grad()
        # Estrai le caratteristiche temporali e statiche
        past_values = past[:, :, 2:].float()
        past_time_features = past[:, :, :2].float()
        future_values = future[:, :, 2:].float()
        future_time_features = future[:, :, :2].float()
        # Maschera di osservazione
        past_observed_mask = torch.ones_like(past_values)
        future_observed_mask = torch.ones_like(future_values)
        # Forward pass
        outputs = model(
            past_values=past_values,
            past_time_features=past_time_features,
            past_observed_mask=past_observed_mask,
            future_time_features=future_time_features
        )
        # Calcola la perdita
        loss = loss_fn(outputs.logits, future_values)
        # Backward pass
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoca {epoch + 1}, Loss: {total_loss / len(dataloader)}')



In [ ]:
model.eval()
with torch.no_grad():
    for past, future in dataloader:
        # Estrai le caratteristiche temporali e statiche
        past_values = past[:, :, 2:].float()
        past_time_features = past[:, :, :2].float()
        future_values = future[:, :, 2:].float()
        future_time_features = future[:, :, :2].float()
        # Maschera di osservazione
        past_observed_mask = torch.ones_like(past_values)
        future_observed_mask = torch.ones_like(future_values)
        # Forward pass
        outputs = model(
            past_values=past_values,
            past_time_features=past_time_features,
            past_observed_mask=past_observed_mask,
            future_time_features=future_time_features
        )
        # Calcola la perdita o altre metriche di valutazione
        loss = loss_fn(outputs.logits, future_values)
        print(f'Loss di valutazione: {loss.item()}')
